# Testing

This Jupyter Notebook is based on Chapter 4 of the book Python for Everybody and Chapters 3 and 6 of the book Think Python.

_(c) 2022, Mark van den Brand and Lina Ochoa Venegas, Eindhoven University of Technology_

## Table of Contents

- [1. Introduction](#introduction)
- [2. Testing](#testing)
- [3. Checking Arguments](#checking-args)
- [4. Manual Testing](#manual-testing)
- [5. Function Testing Guidelines](#testing-guidelines)
- [6. Automated Testing with `docstest`](#doctest)
- [7. The `assert` Statement](#assert)
- [8. Automated Testing with `pytest`](#pytest)
- [9. A Development Plan](#dev-plan)
- [10. Incremental Development](#incremental-dev)
- [11. Refactoring](#refactoring)
- [12. Interface Design](#interface-design)

## 1. Introduction <a class="anchor" id="introduction"></a>

Testing has proven to be essential when building robust programs. 
Although it cannot guarantee that your program is correct, it already increases your confidence that your code behaves as expected (if your test cases pass).
Moreover, it supports evolution and maintainablility of your software, in the sense that if your modify or introduce new code, previous functionality will be tested in the form of a regression test.
However, keeping test up to date and wiring your code with this auxiliary testing code is not trivial.
To solve this issue, there is a plethora of software tools that can be of great help.
In this course we mention two: `doctest` and `pytest`.

## 2. Testing <a class="anchor" id="testing"></a>

**Testing** is a way of convincing yourself that the implementation of the function is correct. 
Testing is not equivalent to giving a formal proof or checking the correcteness of your program.
Actually, testing won't give you any hints on the absence of errors, it will only give you clues about the existence of errors under the cases you have considered.

Whenever you have defined a variable or a function, **verify** it, _before_ using it. 
Do not rely on them blindly.

- In case of a **variable**, inspect
    - type and 
    - value.
- In case of a **function**:
    - choose a set of interesting arguments (e.g., regular, border cases);
    - apply the function to such arguments and gather the results, and;
    - decide on pass or fail.
    
<div class="alert alert-info">
    <b>Testing code in Jupyter notebooks</b><br>
    When testing code in a Jupyter notebook, keep such test code close to the code you are testing, so that you can rerun it and use it as a kind of documentation.
</div>

## 3. Checking Arguments <a class="anchor" id="checking-args"></a>

When calling a function, we have to ensure that the arguments that we pass to the function are *correct*.
Therefore, it is necessary to write auxilary code to perform some checking on the arguments,
to ensure that the given argument does not lead to unwanted behaviour.

In the following example, we define a `countdown` function.
This function gets a number as argument (`nr`) and in every iteration it prints its value and decreases it by `1`.
The loop terminates once the number is equal to `0`.

In [ ]:
def countdown(nr: int) -> None:
    """
    Prints numbers in a decreasing order.
    :param nr: starting countdown number
    """
    while nr != 0:
        print(nr)
        nr -= 1
    print('Done!')

In [ ]:
countdown(15)

In [ ]:
countdown(-15)

In [ ]:
countdown(1.5)

It looks like an infinite computation. How can that be? The function has a condition—when 
`n != 0`. 
But if `n` is not an integer or if it is negative, the condition will never be met!
From there, it gets smaller (more negative), but it will never be 0.

We can solve this problem by checking that the argument passed to the `countdown` function is a positive integer. 

In [ ]:
def countdown(nr: int) -> None:
    """
    Prints numbers in a decreasing order.
    :param nr: starting countdown number
    """
    if not isinstance(nr, int):  # Check that the nr argument is an integer
        print('countdown is only defined for integers.')
    elif nr < 0:                 # Check that the nr argument is positive
        print('countdown is not defined for negative integers.')
    else:                        # If the two conditions are met proceed with the loop
        while (nr != 0):
            print(nr)
            nr -= 1
    print('Done!')

In [ ]:
countdown(-1.5)

The first condition handles non-integers; the second handles negative integers. In both
conditions, the program prints an error message to indicate that something
went wrong.
This program demonstrates the guardin pattern. The first two conditionals
act as guardians, protecting the code that follows from values that might cause an
error. 
The guardians make it possible to prove the correctness of the code.

## 4. Manual Testing <a class="anchor" id="manual-testing"></a>

When thinking about functions, a simple but rather labor-intensive way of testing is manual testing. 
You write in one cell the function body and in a few following cells, the various tests for the function. 

When testing you have to think of corner cases, for instance, empty sequences like strings, the elements at the first or last index of a sequence.
To perform the manual testing of your function, follow the steps below:

1. Choose interesting arguments.
1. Call the function for various arguments in a code cell, such that it shows some result (either via `print` or as last expression in code cell).
1. Visually, check those results.

The combination of the function call plus your chosen arguments is what we call a **test case**.

Let us see an example with the `roll_dice` function.

In [ ]:
import random

def roll_dice(n: int) -> str:
    """
    Rolls a dice n times.
    Assumption: n is an integer and n >= 0.
    
    :param n: number of times to roll the dice
    :returns: a string with the dice number per round (an integer value between 1 and 6).
    """
    rolls: str = ''
    
    for _ in range(n):
        rolls += str(random.randint(1, 6))
    
    return rolls

In [ ]:
# Test case 1: Boundary case (n == 0)
# Expected output: ''
roll_dice(0)

In [ ]:
# Test case 2: Boundary case (n == 100)
# Expected output: string with 100 numbers, each one with a value between 1 and 6
roll_dice(100)

In [ ]:
# Test case 3: Check the length of the resulting string
# Expected output: the length should be equal to the argument
len(roll_dice(3))

In [ ]:
# Test case 4: Check whether all rolls are valid
rolls_valid: bool = True

for roll in roll_dice(10):
    valid: bool = int(roll) in range(1, 6 + 1)
    if not valid:
        rolls_valid = False
        break
    print(f'{roll} in [1, 6]: {valid}')
    
rolls_valid

## 5. Function Testing Guidelines <a class="anchor" id="testing-guidelines"></a>

The challenge with function testing is to convince yourself that you have dealt with all possible cases that the function needs to handle.

* Testing a function in just one call is hardly ever enough.
* Pick a _few_ _important_ arguments, for which you can check the corresponding result.
* Boundary cases, and small typical case
    * Strive for **code coverage**
    * Code that is not executed during the call, is not tested
    * Cover all branches of `if-elif-else`
* You do not need to check the result directly; could test it indirectly by verifying its properties. For instance, by checking the number of elements in a sequence instead of inspecting the sequence.

Suppose you need to write a function that concatenates two strings. 
A sufficient test can be to check whether the length of the resulting string is the same as the length of both string arguments.

## 6. Automated Testing with `doctest` <a class="anchor" id="doctest"></a>

A more robust way of testing is by using automated-testing tools such as `doctest`. 
`doctest` allows you to add **usage examples** to the function docstring.

Below you will find the format of test cases used by `doctest` within your docstring.

```
>>> expression with function call and arguments
expected result
...
...
>>> expression with function call and arguments
expected result
```

<div class="alert alert-info">
    <b>About the expected result in <code>doctest</code></b><br>
    Notice that the <b>expected result</b> is treated as a string that should match the string representation you get by the Python interpreter after running your code.
</div>

In [ ]:
import random

def roll_dice(n: int) -> str:
    """
    Rolls a dice n times.
    Assumption: n is an integer and n >= 0.
    
    :param n: number of times to roll the dice
    :returns: a string with the dice number per round (an integer value between 1 and 6).
    
    Examples and test cases:
    >>> roll_dice(0)                                                 # Test case 1: Boundary case (n == 0)
    ''
    >>> len(roll_dice(3))                                            # Test case 2: Check the length of the resulting string
    3
    >>> all(int(roll) in range(1, 6 + 1) for roll in roll_dice(10))  # Test case 3: Check whether all rolls are valid
    True
    """
    rolls: str = ''
    
    for _ in range(n):
        rolls += str(random.randint(1, 6))
    
    return rolls

### Running test cases for a function with docstring tests?

First, import the `doctest` module.

In [ ]:
import doctest

Second, invoke the `runb_docstring_examples` function. These function **requires two arguments**:
- **`f`:** can be a function, module, or class.
- **`globs`:** a dictionary of arguments used for the execution context. We usually invoke the `globals()` function and we pass it as argument.


Additionally, you can pass other **optional keyword parameters**. In this book we care about two:
- **`verbose=False`:** returns an output only for the failing test cases. If set to `True` the function will return an output even for the passing cases. The default value is `False`.
- **`name='NoName'`:** used in failure messages. We usually set this value to the name of `f` so we can easily identify where we got the error. The default value is `NoName`.

<div class="alert alert-info">
    <b>The <code>runb_docstring_examples</code> function</b><br>
    If you want to learn more about the <code>runb_docstring_examples</code> function, visit <a href=https://docs.python.org/3/library/doctest.html#doctest.run_docstring_examples>this link</a>.
</div>

In [ ]:
doctest.run_docstring_examples(roll_dice, globals(), verbose=True, name='roll_dice')

Same example with `verbose` set to `False`.

In [ ]:
doctest.run_docstring_examples(roll_dice, globals(), verbose=False, name='roll_dice')

Same example without specifying the `name`.

In [ ]:
doctest.run_docstring_examples(roll_dice, globals())

One more example of tests in docstring:

In [ ]:
def find(word: str, letter: str) -> int:
    """
    Finds at which position the letter appears first. If the letter does 
    not appear in the string -1 is returned.
    :param word: base word
    :param letter: letter to find
    :returns: position of the letter within the word.
    
    Examples and test cases:
    >>> isinstance(find('', 'a'), int)  # Test case 1: Check type of find
    True
    >>> find('', 'a')                   # Test case 2: Boundary case (empty string)
    -1
    >>> find('aa', 'a')                 # Test case 3: Boundary case (first position)
    0
    >>> find('bbbba', 'a')              # Test case 4: Boundary case (last position)
    4
    >>> find('bbbbaabbbb', 'a')         # Test case 5: Regular case
    4
    """
    index: int = 0
    
    while index < len(word):
        if word[index] == letter:
            return index
        index += 1
        
    return -1

find('data science', 'a')

In [ ]:
doctest.run_docstring_examples(find, globals(), verbose=True, name='find')

### Running test cases for **all** functions with docstring tests?

The `doctest` module also offers the `testmod` function to run test cases for all functions with docstring tests.
All the parameters of the function are optional.
In this book we care specifically about one: the `verbose` parameter, which has the same semantics or meaning provided for the `run_docstring_examples` function.
By default, this one is set to `False`. 

<div class="alert alert-info">
    <b>The <code>testmod</code> function</b><br>
    If you want to learn more about the <code>testmod</code> function, visit <a href=https://docs.python.org/3/library/doctest.html#doctest.testmod>this link</a>.
</div>

In [ ]:
doctest.testmod(verbose=True)  # with details

In [ ]:
doctest.testmod(verbose=False)  # without details

## 7. The `assert` Statement <a class="anchor" id="assert"></a>

An **assertion** is a statement that checks if a Boolean expression returns `True`.
If `True` the execution of the program continues, otherwise the program raises an `AssertionError` with an optional error message.
The syntax of the statement looks as follows.

```python
# First option without message
assert <boolean_condition>

# Second option with message
assert <boolean_condition>, <error_message>
```

Assertions are used as internal checks within your program to verify that certain preconditions are being met and as a debugging tool.
They are not supposed to be used as a way to communicate with end users, but more as an internal check when developing a program.

<div class="alert alert-info">
    <b>Inform about an error via an exception</b><br>
    If you need to inform the user about a mistake that comes from their side, then you will need to raise an exception.
</div>

For instance, if we go back to the `roll_dice` example, we can use assertions to check if the argument is a positive integer. 

In [ ]:
import random

def roll_dice(n: int) -> str:
    """
    Rolls a dice n times.
    Assumption: n is an integer and n >= 0.
    
    :param n: number of times to roll the dice
    :returns: a string with the dice number per round (an integer value between 1 and 6).
    """
    assert isinstance(n, int), 'n must be an integer'
    assert n >= 0, 'n must be a positive number'
    
    rolls: str = ''
    
    for _ in range(n):
        rolls += str(random.randint(1, 6))
    
    return rolls

In [ ]:
# Trigger first assertion error (not an integer)
roll_dice('10')

In [ ]:
# Trigger second assertion error (negative number)
roll_dice(-10)

## 8. Automated Testing with `pytest` <a class="anchor" id="pytest"></a>

[`pytest`](https://docs.pytest.org/en/7.1.x/index.html) is a Python testing framework that supports the creation of more robust tests and speeds up the testing process by providing boilerplate code (repetitive and common code used in several parts of your program) that you can reuse for testing purposes.
It is usually used when dealing with Python projects but new modules have been developed to integrate them with Jupyter notebooks.
In this book, we introduce them so you can have an idea of how to interact with this module, especially when writting Python projects outside of the notebooks environment.

### Installing `pytest` and `ipytest`
To use `pytest` install the module via Anaconda or the terminal with the command `pip install pytest`.
To use it in combination with Jupyter notebooks, you will need also to install the `ipytest` module in the same way you did it with `pytest`.
[`ipytest`](https://github.com/chmp/ipytest) is a library that supports the execution of `pytest` in the Jupyter Notebook environment.
This is an open-source project supported by a small community of developers.

<div class="alert alert-info">
    <b>Risks of some open-source modules</b><br>
    There are some risks attached to the use of open-source modules with a small community behind them:
    <ul>
        <li>Maintenance happens irregularly.</li>
        <li>There is a chance that the project becomes inactive if there is no funding coming in or if the collaborators lose interest.</li>
        <li>Resources are limited, thus the testing and robustness of the project might be affected.</li>
    </ul>
</div>

Given the current maturity of the tools, you need to be careful when testing your code in a Jupyter notebook with `pytetst`.
`pytest` is a robust library but the wrappers that connect it with the Jupyter Notebook environment might need some more time to mature.
The ideal setting to use `pytest` is during the development of stand-alone Python projects.

### Creating the test cases

To create test cases with `pytest` you just need to define a new function that starts with the name "test_".
You need to pick your own conventions to name the test case after the prefix we have provided.

Let us go back to the `find` example and translate our `doctest` cases into `pytest` cases.

In [ ]:
def find(word: str, letter: str) -> int:
    """
    Finds at which position the letter appears first. If the letter does 
    not appear in the string -1 is returned.
    :param word: base word
    :param letter: letter to find
    :returns: position of the letter within the word.
    """
    index: int = 0
    
    while index < len(word):
        if word[index] == letter:
            return index
        index += 1
        
    return -1

find('data science', 'a')

Below you will find our test cases.
The convention we picked to name them is as follows: `test_<function>_<test_case>`.
Notice that to perform the checks, we use the `assert` statement!

In [ ]:
def test_find_check_type() -> None:
    assert isinstance(find('', 'a'), int), \
    'The return type of `find` must be an integer'
    
def test_find_empty_string() -> None:
    assert find('', 'a') == -1, \
    '`find` does not return -1 when passing an empty string as parameter'
    
def test_find_first_position() -> None:
    assert find('aa', 'a') == 0, \
    '`find` does not return the index of the first appearance of the letter'
    
def test_find_last_position() -> None:
    assert find('bbbba', 'a') == len('bbbba') - 1, \
    '`find` does not return the index when the letter is in the last position'
    
def test_find_regular_case() -> None:
    assert find('bbbbaabbbb', 'a') == 4,\
    '`find` does not return the right index of the letter'

### Running the test cases

To run `pytest` in this Jupyter notebooks, we will need first to import `ipytest` and then invoke the `autoconfig` function.
The latter will configure `ipytest` with default values.
Then, you will be able to invoke the `run` function in charge of executing all tests.

In [ ]:
import ipytest

ipytest.autoconfig()  # Configures ipytest
ipytest.run()         # Runs all tests within the notebook

### Working with fixtures

If you want to save certain values and avoid invoking the same functions with the same arguments, you can create *fixtures*.
**Fixtures** are functions that create data or initializes the state of a program.
Test can use fixtures to avoid duplicating code!
To use them they need to explicitly refer to them, and pass the function as an argument!
The syntax is as follows:

```python
import pytest

@pytest.fixture
def <fixture_name>():
    # Body
    return <value>

def test_<test_name>(<fixture_name>):
    assert <fixture_name> == ...
```

Let us see modify our find tests to see how fixtures work.

In [ ]:
import pytest
import ipytest
ipytest.clean_tests()
ipytest.autoconfig()

@pytest.fixture
def regular_case() -> int:
    return find('bbbbaabbbb', 'a')


def test_find_check_type(regular_case) -> None:
    assert isinstance(regular_case, int), \
    'The return type of `find` must be an integer'

def test_find_regular_case(regular_case) -> None:
    assert regular_case == 4,\
    '`find` does not return the right index of the letter'
    
ipytest.run()

Notice that if you want to ignore previously ran tests, you need to explicitly invoke the function `clean_tests`, which will remove all tests whose name starts by "Test" or "test".

<div class="alert alert-info">
    <b><code>ipytest</code> documentation</b><br>
    If you want to learn more about the <code>ipytest</code> functions, please have a look at the official documentation <a href=https://github.com/chmp/ipytest#ipytestconfig>here</a>.
</div>

## 9. A Development Plan <a class="anchor" id="dev-plan"></a>

A development plan is a process for writing programs. 

The process we use is “encapsulation and generalization”. 

The steps of this process are:
1. Start by writing a small program with no function definitions.
2. Once you get the program working, identify a coherent piece of it, encapsulate the
piece in a function and give it a name.
3. Generalize the function by adding appropriate parameters.
4. Repeat steps 1–3 until you have a set of working functions. Copy and paste working
code to avoid retyping (and re-debugging).
5. Look for opportunities to improve the program by refactoring. For example, if you
have similar code in several places, consider factoring it into an appropriately general
function.

This process has some drawbacks—we will see alternatives later—but it can be useful if
you do not know ahead of time how to divide the program into functions. This approach
lets you design as you go along.

## 10. Incremental Development <a class="anchor" id="incremental-dev"></a>

The larger the program you need to develop the more useful it is to have a structured way of developing it.
There are multiple ways of structuring the development process and in fact your computational thinking. 

An important development paradigm is **divide and conquer**. You can, for example, try to split the problem in subproblems and solve each subproblem in isolation. This paradigm works for large and complex problems, furthermore, it allows parallel development of a program.

However, before applying any development paradigm you need to understand the problem. Critical reading is essential, making sketches to visualise the problem and the solution, explain the solution to somebody else, and finally write the code.

Another paradigm, as advocated by "Think Python" is to use **incremental development**.
The idea is not to try develop a complete program in one go, but to develop it step by step.
Along the way you can write small tests to see whether you program behaves correctly.
Developing a complete program is likely to fail and you will spend a lot of time on debugging.


As an example, suppose you want to calculate the volume of a hollow cylinder, so a cylinder with an inner cylinder removed. 
The problem we need to solve is subtracting the volume of the inner cylinder from the volume of the outer cylinder. 
The volume of a cylinder is calculated as $volume = h*\pi*R^2$.
Can we now mathematical formulate the problem?

$volume = h (\pi R^2 - \pi r^2)$


<img src="assets/hollow-cylinder.png"/>

The first step is to consider how a function to calculate the volume of a hollow cylinder should look like in Python. 
In other words, what are the inputs (parameters) and what is the output (return value)?
A first try is to write a function with three numbers, which represent the radius of
the two cylinders and their heights, the result is a floating-point value.

In [ ]:
def volume(outer_radius: int, inner_radius: int, height: int) -> float:
    """
    Computers the volume of a hollow cylinder based on the radius 
    of the outer and inner cylinder.
    :param outer_radius: outer radius of the cylinder
    :param inner_radius: inner radius of the cylinder
    :param height: height of the cylinder
    :returns volume of a hollow cylinder.
    """
    return 0.0

volume(3, 1, 3)

This function is obviously not correct, it does not calculate the value of a hollow cylinder, it just returns `0.0`.
However, we have now a skeleton of the function.
So, in the next step we can start developing the body of the function.

We need first to calculate the volume of a single cylinder based on the radius and
the height. 
We introduce two auxilary variables `surface_c1` and `surface_c2`.
Furthermore we add a `print` statement to see whether the values make sense.

In [ ]:
import math

def volume(outer_radius: int, inner_radius: int, height: int) -> float:
    """
    Computers the volume of a hollow cylinder based on the radius 
    of the outer and inner cylinder.
    :param outer_radius: outer radius of the cylinder
    :param inner_radius: inner radius of the cylinder
    :param height: height of the cylinder
    :returns volume of a hollow cylinder.
    """
    surface_c1: float = math.pi * outer_radius**2
    surface_c2: float = math.pi * inner_radius**2
    print('surface_c1 is', surface_c1)
    print('surface_c2 is', surface_c2)
    
    return 0.0

volume(3, 1, 4)

The next step is to calculate the difference of the surfaces.

In [ ]:
import math

def volume(outer_radius: int, inner_radius: int, height: int) -> float:
    """
    Computers the volume of a hollow cylinder based on the radius 
    of the outer and inner cylinder.
    :param outer_radius: outer radius of the cylinder
    :param inner_radius: inner radius of the cylinder
    :param height: height of the cylinder
    :returns volume of a hollow cylinder.
    """
    surface_c1: float = math.pi * outer_radius**2
    surface_c2: float = math.pi * inner_radius**2
    diff_surfaces = surface_c1 - surface_c2
    print('diff_surfaces is', diff_surfaces)
    
    return 0.0

volume(3, 1, 4)

Then, we add a statement for multiplying the obtained difference with the height.

In [ ]:
import math

def volume(outer_radius: int, inner_radius: int, height: int) -> float:
    """
    Computers the volume of a hollow cylinder based on the radius 
    of the outer and inner cylinder.
    :param outer_radius: outer radius of the cylinder
    :param inner_radius: inner radius of the cylinder
    :param height: height of the cylinder
    :returns volume of a hollow cylinder.
    """
    surface_c1: float = math.pi * outer_radius**2
    surface_c2: float = math.pi * inner_radius**2
    diff_surfaces: float = surface_c1 - surface_c2
    print('diff_surfaces is', diff_surfaces)
        
    return height * diff_surfaces

volume(3, 1, 4)

The last step is to make sure that if the outer radius is smaller than the inner radius. Thus, we return `0` instead of a negative volume and print an error message.

In [ ]:
import math

def volume(outer_radius: int, inner_radius: int, height: int) -> float:
    """
    Computers the volume of a hollow cylinder based on the radius 
    of the outer and inner cylinder, using outer_radius >= inner_radius.
    :param outer_radius: outer radius of the cylinder
    :param inner_radius: inner radius of the cylinder
    :param height: height of the cylinder
    :returns volume of a hollow cylinder.
    """
    if outer_radius >= inner_radius:
        surface_c1: float = math.pi * outer_radius**2
        surface_c2: float = math.pi * inner_radius**2
        diff_surfaces: float = surface_c1 - surface_c2
        
        volume_cylinder: float = height * diff_surfaces
    else:
        print("The outer cylinder should be larger than the inner cylinder")
        
        volume_cylinder: float = 0
        
    return volume_cylinder

volume(3, 1, 4)

The final version of the function does not display anything when it runs; it only returns
a value, or prints an error message.
The `print` statements we wrote are useful for debugging, but once you get the
function working, you should remove them. 
Code like that is called **scaffolding** because it
is helpful for building the program but is not part of the final product.

When you get more experienced, you will add more statements in one run and use less debugging statements.
The key points of the incremental approach are:

1. Start with a working program and make small incremental changes. At any point, if
there is an error, you should have a good idea where it is.
2. Use variables to hold intermediate values so you can display and check them.
3. Once the program is working, you might want to remove some of the scaffolding or
consolidate multiple statements into compound expressions, but only if it does not
make the program difficult to read.

## 11. Refactoring <a class="anchor" id="refactoring"></a>

After you have written a program, it makes sense to study the code and see whether there are opportunities to improve the structure of the code. 

If you have copied and pasted instructions, it is advisable to introduced a separate function.

It may also be the case that you developed a function with a complex and bulky body. Then, consider the introduction of auxilary functions.

Or maybe you have some variables or functions that have a name that is not very descriptive of what the

All these improvements are **refactorings**. 

Once you start coding, you understand the problem better. 
Sometimes refactoring is a sign that you have learned something.

## 12. Interface Design <a class="anchor" id="interface-design"></a>

It may not be clear why it is worth the trouble to divide a program into functions. There
are several reasons:

* Creating a new function gives you an opportunity to name a group of statements,
which makes your program easier to read and debug.
* Functions can make a program smaller by eliminating repetitive code. Later, if you
make a change, you only have to make it in one place.
* Dividing a long program into functions allows you to debug the parts one at a time
and then assemble them into a working whole.
* Well-designed functions are often useful for many programs. Once you write and
debug one, you can reuse it.

The **interface** of a function is a summary of how it is used: 
* What are the parameters and their types? 
* What does the function do? 
* What is the return value and its type, if any? 

An interface is “clean” if it allows the caller to do what they want without dealing with unnecessary details of
the body of the function.
A good interface can be defined by choosing a good name for the function and for its parameters and by providing the types of its arguments and result.
A short description, via comments, of the basic functionality of the function is also useful. This description can include the restrictions (so-called **pre-conditions**) for calling the function.

---

# (End of Notebook)

&copy; 2022-2023 - **TU/e** - Eindhoven University of Technology